In [31]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import time

In [32]:
#column-1 contains the labels, so cast it to int
#other columns are pixel intensities, so cast them to floating-point numbers
df = pd.read_csv("train.csv")
train = df.as_matrix()

#intensities
train_y = train[:,0].astype('int8')
train_x = train[:,1:].astype('float64')

train = None

test = pd.read_csv("test.csv").as_matrix().astype('float64')

In [33]:
# note that we will have bad accuracy if the examples for each digit are not uniformly distributed.
#So check if they are.
no_of_train = train_y.shape[0]
counts = [0]*10
for i in range(no_of_train):
    counts[train_y[i]] += 1
for i in counts:
    print(i)

4132
4684
4177
4351
4072
3795
4137
4401
4063
4188


Almost Uniform. So apply NN happily!

In [34]:
# Normalise to keep floating-point underflow in check!
test /= 255
train_x /= 255

In [35]:
#look at this : https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html
#gives you one-hot encoding.
train_y = pd.get_dummies(train_y).as_matrix()

As one-hot encoding is used here and we have to predict from 10 labels ie: the most 'probable', makes sense to use Softmax function : 
\begin{equation*}
y=σ(x)=\frac{e^x}{\sum_{i=1}^Ne^x}
\end{equation*}

Recall that the derivative of sigmoid is :
\begin{equation*}
y=σ(x)=y*(1-y)
\end{equation*}
I intend to apply the most-widely used combination : **Cross-entropy as Loss-function + Softmax classification + weight-decay**. (Might apply RBMs in another notebook on MNIST, to learn the same!). Read this : http://www.cs.toronto.edu/~hinton/absps/fastnc.pdf

No. of hidden layers : 2 with 500 and 300 units in each respectively.

In [36]:
#randomly initialised using : random gaussian function with mean=zero and variance between 1/sqtr(num_inputs_layer)
def getWeights():
    inp = 784
    w1_dim = 500
    w2_dim = 300
    classes = 10
    
    #layer1
    w1 = np.random.normal(0,inp**-0.5,[inp,w1_dim])
    b1 = np.random.normal(0,inp**-0.5,[1,w1_dim])
    
    #layer2
    w2 = np.random.normal(0,w1_dim**-0.5,[w1_dim,w2_dim])
    b2 = np.random.normal(0,w1_dim**-0.5,[1,w2_dim])
    
    #layer3
    w3 = np.random.normal(0,w2_dim**-0.5,[w2_dim,classes])
    b3 = np.random.normal(0,w2_dim**-0.5,[1,classes])
    
    return [w1,b1,w2,b2,w3,b3]

Read this : http://jmlr.org/papers/volume15/srivastava14a.old/srivastava14a.pdf to learn more about 'Dropout'. It is mainly to avoid overfitting by ignoring a few input units. Here, 'p' is the probability that a unit will be ignored. So simply zero out the selected ones. And n testing, the entire network is considered and each activation is reduced by a factor p. https://medium.com/@amarbudhiraja/https-medium-com-amarbudhiraja-learning-less-to-learn-better-dropout-in-deep-machine-learning-74334da4bfc5 : see this for a condensed version of the paper.

In [64]:
def softmax(x):
    return (np.exp(x).T/np.sum(np.exp(x),axis=1)).T

def ReLu(x,der=False):
    if der==False:
        return x*(x>0)
    else:
        return 1*(x>0)

def dropout(x,p):
    m = np.random.binomial([np.ones_like(x)],(1-p))[0] / (1-p)
    #note that only the first dimension is being taken.
    return x*m
                           
def forward_prop(x,weights,p=0):
    w1,b1,w2,b2,w3,b3 = weights                    

    #in case you wondering : https://stackoverflow.com/questions/27385633/what-is-the-symbol-for-in-python
    z1 = ReLu(x@w1 + b1)
    z1 = dropout(z1,p)
                           
    z2 = ReLu(z1@w2 + b2)
    z2 = dropout(z2,p)
                           
    return [z1,z2,softmax(z2@w3 + b3)]

Use the cross-entropy loss function when dealing with classification:
\begin{equation*}
-\frac{1}{N}.{\sum_{i=1}^N[y*log(a)+(1-y)log(1-a)]}
\end{equation*}

Here is a refresher(I knew you'd forget!): https://github.com/Kulbear/deep-learning-nano-foundation/wiki/ReLU-and-Softmax-Activation-Functions

In [38]:
def log2(x):
    if x!=0:
        return np.log(x)
    else:
        return -np.inf #don't worry, taken care by nan_to_num : 
    #https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.nan_to_num.html
    
def log(y):
    return [[log2(nx) for nx in x] for x in y]

def cost(a,y):
    loss = -np.mean((np.nan_to_num(y*log(a)) + np.nan_to_num((1-y)*log(1-a))),keepdims = True)
    return loss

In [39]:
#add cross-validation
frac = 0.2
sz = round(train_x.shape[0]*frac)

indices = np.arange(no_of_train)
np.random.shuffle(indices)

x_train = train_x[indices[sz:]]
x_valid = train_x[indices[:sz]]

y_train = train_y[indices[sz:]]
y_valid = train_y[indices[:sz]]

train_x = None
train_y = None

x_train.shape

(33600, 784)

And here it comes : Back Prop!! Just apply chain-rule repeatedly to arrive at those complex equations that we got in Stanford ML course! 
For output layer : 
\begin{equation*}
\nabla W_j=\frac{dL(W_{ij})}{dW_{ij}} = \frac{1}{N}.(t-y)X_{ij}
\end{equation*}

For hidden layer :
\begin{equation*}
\nabla W_j=\frac{dL(W_{ij})}{dW_{ij}} =  \frac{1}{N}.(t-y).W_{ij}.\sigma(x)^{'}X_{ij}
\end{equation*}
Enter Momentum! Used to avoid getting trapped in local minima by taking leaps and decreases convergence time. Read this : https://www.quora.com/What-does-momentum-mean-in-neural-networks
\begin{equation*}
V_{i+1} = \gamma V_{i} + \eta.\nabla L(W) \\
W = W - V_{i+1}
\end{equation*}
Also, add regularisation using L2 norm.
\begin{equation*}
L = -\frac{1}{N}.{\sum_{i=1}^N[ylog(a)+(1-y)log(1-a)]} + \frac{1}{2}.\lambda.\sum_{j=1}^{nj}\sum_{i=1}^{ni}W_{ij}^{2}
\end{equation*}
As usual, you can't not look at this : http://neuralnetworksanddeeplearning.com/chap2.html

In [58]:
def SGD(weights,x,y,op,eta_lr,gamma_mom,lbda_reg,cache = None):
    w1,b1,w2,b2,w3,b3 = weights
    if cache == None:
        v_w1 = np.zeros_like(w1)
        v_w2 = np.zeros_like(w2)
        v_w3 = np.zeros_like(w3)
        v_b1 = np.zeros_like(b1)
        v_b2 = np.zeros_like(b2)
        v_b3 = np.zeros_like(b3)
    
    else:
        v_w1,v_b1,v_w2,v_b2,v_w3,v_b3 = cache
    
    z1,z2,a = op
    w3_delta = (y-a)/x.shape[0]
    w2_error = w3_delta@w3.T
    w2_delta = w2_error*ReLu(z2,der=True)
    w1_error = w2_delta@w2.T
    w1_delta = w1_error*ReLu(z1,der=True)
    
    eta_lr = -eta_lr
    #print(w3_delta.shape,w2_delta.shape,w1_delta.shape)
    #look at the last link in the previous cell for the clearest explanation for the following expressions.
    #incorporate l2 regularisation, too.
    v_w3 = gamma_mom*v_w3 + eta_lr*(z2.T@w3_delta + lbda_reg*w3)
    v_b3 = gamma_mom*v_b3 + eta_lr*(w3_delta.sum(axis=0) + lbda_reg*b3)
    
    v_w2 = gamma_mom*v_w2 + eta_lr*(z1.T@w2_delta + lbda_reg*w2)
    v_b2 = gamma_mom*v_b2 + eta_lr*(w2_delta.sum(axis=0) + lbda_reg*b2)
    
    v_w1 = gamma_mom*v_w1 + eta_lr*(x.T@w1_delta + lbda_reg*w1)
    v_b1 = gamma_mom*v_b1 + eta_lr*(w1_delta.sum(axis=0) + lbda_reg*b1)

    w3 -= v_w3
    b3 -= v_b3
    
    w2 -= v_w2
    b2 -= v_b2
    
    w1 -= v_w1
    b1 -= v_b1
    
    weights = [w1,b1,w2,b2,w3,b3]
    cache = [v_w1,v_b1,v_w2,v_b2,v_w3,v_b3]
    
    return weights,cache

This is to improve the performance of the neural network. Taken from : https://gist.github.com/fmder/e28813c1e8721830ff9c

In [41]:
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

def elastic_transform(image, alpha, sigma, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))

    return map_coordinates(image, indices, order=1).reshape(shape)

In [66]:
def accuracy(pred,y):
    tot = 0
    n = pred.shape[0]
    pred = np.argmax(pred,axis = 1)
    y = np.argmax(y,axis = 1)
    for i in zip(pred,y):
        if i[0] == i[1]:
            tot += 1 
    return (tot*100)/n
    
def run(weights,x_train,y_train,x_valid,y_valid,epochs = 10,batch_sz = 25,eta = 1e-3,
        decay = 0,momentum=0,l2=0.001,p=0):
    cache = None
    history = [[],[]] 
    mtime = 0
    index = np.arange(x_train.shape[0])
    for j in range(epochs):
        t = 0
        np.random.shuffle(index)
        iterations = round(x_train.shape[0]/batch_sz)
        sacc = 0
        sloss = 0
        print("Epoch: {0} / {1}\n".format(j+1,epochs))
        stime = 0
        timeIT = time.time()
        for i in range(iterations):
            timeI = time.time()
            f = i*batch_sz
            l = f + batch_sz
            #all this to take care when the number of samples is not exaftly divisible by batch_sz
            if l>x_train.shape[0]-1 :
                l = x_train.shape[0]
            x = np.array([elastic_transform(xx.reshape(28,28),15,3).reshape(784) for xx in x_train[index[f:l]]])
            y = y_train[index[f:l]]
            outputs = forward_prop(x,weights,p)
            loss = cost(outputs[-1],y)
            accuracy_t = accuracy(outputs[-1],y)
            sacc += accuracy_t
            sloss += loss
            accuracy_train = sacc/(i+1)
            loss_train = sloss/(i+1)
            #SGD(weights,x,y,op,eta_lr,gamma_mom,lbda_reg,cache = None)
            weights,cache = SGD(weights,x,y,outputs,eta,momentum,l2,cache)
            t += x.shape[0]
            stime += time.time() - timeI
            mtime = stime/(i+1)
            mTimeT = mtime * (iterations-i-1)
            sys.stdout.write("\r%5d/%5d ETA: %3d s - loss : %.4f acc : %.4f" % (t,x_train\
            .shape[0],mTimeT,loss_train,accuracy_train))
            history[0].append([loss_train,accuracy_train])
        mtime = time.time()-timeIT
        eta = eta - (eta*decay)
        
        outputs = forward_prop( x_valid,weights)
        
        loss_valid = cost(outputs[-1], y_valid)
        accuracy_valid = accuracy(outputs[-1], y_valid)
        sys.stdout.write("\r%5d/%5d ETA: %3d s loss: %.4f  acc: %.4f - lossValid: %.4f  accValid: %.4f " % ( t, x_train.shape[0],
        mtime, loss_train, accuracy_train, loss_valid, accuracy_valid))
        history[1].append([loss_valid, accuracy_valid])
    return weights,history

In [ ]:
weights = getWeights()

alpha = 1e-1
epochs = 30
nbatchs = 100
weights, history = run(weights, 
              x_train, y_train, 
              x_valid, y_valid, 
              epochs = epochs,
              batch_sz=nbatchs, 
              eta = alpha, 
              decay = 0.1, 
              momentum = 0.9, 
              l2 = 1e-7, 
              p = 0.25)

Epoch: 1 / 30

33600/33600 ETA:  61 s loss: 0.0954  acc: 81.2113 - lossValid: 0.0278  accValid: 94.9048 Epoch: 2 / 30

33600/33600 ETA:  61 s loss: 0.0545  acc: 90.1101 - lossValid: 0.0210  accValid: 96.3095 Epoch: 3 / 30

33600/33600 ETA:  61 s loss: 0.0444  acc: 91.9881 - lossValid: 0.0178  accValid: 96.9405 Epoch: 4 / 30

33600/33600 ETA:  61 s loss: 0.0382  acc: 93.4107 - lossValid: 0.0150  accValid: 97.4643 Epoch: 5 / 30

33600/33600 ETA:  61 s loss: 0.0342  acc: 93.8363 - lossValid: 0.0134  accValid: 97.7738 Epoch: 6 / 30

33600/33600 ETA:  61 s loss: 0.0322  acc: 94.2321 - lossValid: 0.0126  accValid: 97.8929 Epoch: 7 / 30

33600/33600 ETA:  61 s loss: 0.0304  acc: 94.8214 - lossValid: 0.0114  accValid: 98.1310 Epoch: 8 / 30

33600/33600 ETA:  62 s loss: 0.0282  acc: 95.0833 - lossValid: 0.0106  accValid: 98.1310 Epoch: 9 / 30

33600/33600 ETA:  61 s loss: 0.0275  acc: 94.9792 - lossValid: 0.0102  accValid: 98.2619 Epoch: 10 / 30

33600/33600 ETA:  62 s loss: 0.0261  acc: 95.330